In [1]:
from ytmusicapi import YTMusic as yt
import re
import pandas as pd
import requests as rq
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import time
import spotipy.util as util
import fuzzywuzzy as fw
from fuzzywuzzy import fuzz
import numpy as np

In [2]:
ytmusic = yt('headers_auth.json')

In [3]:
user_profile = input("Enter the Library link of your profile: \n")
user_yt = user_profile.split('/')[-1]

Enter the Library link of your profile: 
https://music.youtube.com/channel/UCLcvWDLANmp4g4dWOdMCoVg


In [4]:
user_yt

'UCLcvWDLANmp4g4dWOdMCoVg'

In [5]:
playlist_link = input("Enter the Playlist link of your profile: \n")
playlist_yt = playlist_link.split("=")[-1]

Enter the Playlist link of your profile: 
https://music.youtube.com/playlist?list=PLLT7phSFVy-lfNrRTXR_A_HiIpjL5IAEz


In [6]:
playlist_yt

'PLLT7phSFVy-lfNrRTXR_A_HiIpjL5IAEz'

In [7]:
playlist_yt_details = ytmusic.get_playlist(playlist_yt, limit = 1000)

In [8]:
playlist_yt_details

{'id': 'PLLT7phSFVy-lfNrRTXR_A_HiIpjL5IAEz',
 'privacy': 'PUBLIC',
 'title': 'Bridge Over Troubled Life',
 'thumbnails': [{'url': 'https://yt3.ggpht.com/H4_Lp8ZIprNkq_ZJSxyqNyv8D6hQ46cQehEuIjCnfrl2UAtGfF21WpbuQH4UmQqu5xVX7txzk3F3=s192',
   'width': 192,
   'height': 192},
  {'url': 'https://yt3.ggpht.com/H4_Lp8ZIprNkq_ZJSxyqNyv8D6hQ46cQehEuIjCnfrl2UAtGfF21WpbuQH4UmQqu5xVX7txzk3F3=s576',
   'width': 576,
   'height': 576},
  {'url': 'https://yt3.ggpht.com/H4_Lp8ZIprNkq_ZJSxyqNyv8D6hQ46cQehEuIjCnfrl2UAtGfF21WpbuQH4UmQqu5xVX7txzk3F3=s1200',
   'width': 1200,
   'height': 1200}],
 'author': {'name': 'Ruchit Tripathi', 'id': 'UCLcvWDLANmp4g4dWOdMCoVg'},
 'duration': '6+ hours',
 'trackCount': 187,
 'tracks': [{'videoId': 'HvropLxYb5c',
   'title': 'Footloose (From "Footloose" Soundtrack)',
   'artists': [{'name': 'Kenny Loggins', 'id': 'UCF-DjS6Y9CKZThQf01SYe7w'}],
   'album': {'name': "Footloose (15th Anniversary Collectors' Edition)",
    'id': 'MPREb_04te88CNvtj'},
   'likeStatus': 'INDI

In [9]:
tracks_yt = playlist_yt_details['tracks']

In [10]:
tracks_yt

[{'videoId': 'HvropLxYb5c',
  'title': 'Footloose (From "Footloose" Soundtrack)',
  'artists': [{'name': 'Kenny Loggins', 'id': 'UCF-DjS6Y9CKZThQf01SYe7w'}],
  'album': {'name': "Footloose (15th Anniversary Collectors' Edition)",
   'id': 'MPREb_04te88CNvtj'},
  'likeStatus': 'INDIFFERENT',
  'thumbnails': [{'url': 'https://lh3.googleusercontent.com/sYwKGkGzYd0eczgsbRwd6-OWhWSJY1j5jWMWI05GDbxb8pMfOHWH2qXYciOr0Ankzq0BYGW2QDSMeJYH=w60-h60-l90-rj',
    'width': 60,
    'height': 60},
   {'url': 'https://lh3.googleusercontent.com/sYwKGkGzYd0eczgsbRwd6-OWhWSJY1j5jWMWI05GDbxb8pMfOHWH2qXYciOr0Ankzq0BYGW2QDSMeJYH=w120-h120-l90-rj',
    'width': 120,
    'height': 120}],
  'isAvailable': True,
  'isExplicit': False,
  'duration': '3:47',
  'setVideoId': '56B44F6D10557CC6',
  'feedbackTokens': {'add': 'AB9zfpLVm9Ue2Fo-4uJ7xjd3EyS4OkvvCXmBKpHMD668DZb_X-8gZu89caUku3ctGroAj0ucYdzuGdBPzEd5RYzZGOGCoHPQfQ',
   'remove': 'AB9zfpI8d8pKH0GgryR5xQ9ooEZsoTzMhIF2BNUhaAYoPlvWt4u_3WDSANJoUMNuAM5pgaa08-Qa-UlRG

In [11]:
len(tracks_yt)

187

In [12]:
list_yt_tracks = []

In [13]:
for tracks in tracks_yt:
    list_yt_tracks.append({'title':tracks['title'],'artist':tracks['artists'][0]['name'],'album':('None' if tracks['album'] is None else tracks['album']['name']),'duration':(tracks['duration'] if 'duration' in tracks else 'None')})

In [14]:
len(list_yt_tracks)

187

In [15]:
df_yt_tracks = pd.DataFrame(list_yt_tracks)

In [16]:
df_yt_tracks.loc[(df_yt_tracks['duration'] == 'None')]

,title,artist,album,duration


In [17]:
auth_mgr = SpotifyClientCredentials(client_id = "f23579cc015b4e518158755fd63d46fc", client_secret = "9dd9fb72962e4c3f9d42474c0142313a")

In [41]:
client_id = "f23579cc015b4e518158755fd63d46fc"
client_secret = "9dd9fb72962e4c3f9d42474c0142313a"
username = 'v52avba3w6pkkxosgeglngh2c'
scope = 'playlist-modify-public'

In [42]:
auth_mgr = util.prompt_for_user_token(username,scope,client_id=client_id,client_secret=client_secret,redirect_uri='http://localhost:8888/') 

In [43]:
sp = spotipy.Spotify(auth = auth_mgr)

In [20]:
playlist_name = 'Test Playlist'
des = "Importing songs from Youtube Music, Bye Bye YT music for 6 months! :P"

In [21]:
sp.user_playlist_create('v52avba3w6pkkxosgeglngh2c',playlist_name, description = des)

{'collaborative': False,
 'description': 'Importing songs from Youtube Music, Bye Bye YT music for 6 months! :P',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/3Rmmf5dVAIywEUhbIhndcf'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/3Rmmf5dVAIywEUhbIhndcf',
 'id': '3Rmmf5dVAIywEUhbIhndcf',
 'images': [],
 'name': 'Test Playlist',
 'owner': {'display_name': 'Ruchit Tripathi ',
  'external_urls': {'spotify': 'https://open.spotify.com/user/v52avba3w6pkkxosgeglngh2c'},
  'href': 'https://api.spotify.com/v1/users/v52avba3w6pkkxosgeglngh2c',
  'id': 'v52avba3w6pkkxosgeglngh2c',
  'type': 'user',
  'uri': 'spotify:user:v52avba3w6pkkxosgeglngh2c'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSwzZGYxN2JkODNhMzZmMTE2NWE2YjZlYWE5YTM5YjlmN2MyODNmYWE5',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/3Rmmf5dVAIywEUhbIhndcf/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
 

In [22]:
sp.current_user()

{'display_name': 'Ruchit Tripathi ',
 'external_urls': {'spotify': 'https://open.spotify.com/user/v52avba3w6pkkxosgeglngh2c'},
 'followers': {'href': None, 'total': 2},
 'href': 'https://api.spotify.com/v1/users/v52avba3w6pkkxosgeglngh2c',
 'id': 'v52avba3w6pkkxosgeglngh2c',
 'images': [{'height': None,
   'url': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=228232451258647&height=300&width=300&ext=1619233069&hash=AeTSlbKQTP98V-L4FhU',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:v52avba3w6pkkxosgeglngh2c'}

In [23]:
df_yt_tracks.columns

Index(['title', 'artist', 'album', 'duration'], dtype='object')

In [22]:
result = sp.search(q= f"{df_yt_tracks['title'].iloc[0]} {df_yt_tracks['artist'].iloc[0]} ")

In [23]:
result = sp.search(q=(df_yt_tracks['title'].iloc[0]+''+df_yt_tracks['artist'].iloc[0]), limit = 5, type = 'track')

In [24]:
result['tracks']['total']

5

In [25]:
result['tracks']['next']

In [27]:
df_yt_tracks['title'].iloc[0]

'Footloose (From "Footloose" Soundtrack)'

In [246]:
df_yt_tracks['artist'] = df_yt_tracks['artist'].apply(lambda x:re.sub(r'\([^()]*\)','',x))

In [247]:
df_yt_tracks['artist'] = df_yt_tracks['artist'].apply(lambda x:x.capitalize())

In [248]:
df_yt_tracks['artist']

0          Kenny loggins
1             Billy joel
2      Daryl braithwaite
3           Van morrison
4             Elton john
             ...        
182           Air supply
183           Imran khan
184            Vance joy
185            James bay
186             E2 ielts
Name: artist, Length: 187, dtype: object

In [20]:
df_yt_tracks['title'] = df_yt_tracks['title'].apply(lambda x:re.sub(r'\([^()]*\)','',x))

In [21]:
df_yt_tracks['title'] = df_yt_tracks['title'].apply(lambda x:x.strip())

In [22]:
df_yt_tracks['title']

0                      Footloose
1           Just the Way You Are
2                     One Summer
3                Brown Eyed Girl
4                      Sacrifice
                 ...            
182              All Out of Love
183                     Satisfya
184                 Mess Is Mine
185                    Let It Go
186    How to Prepare for IELTS!
Name: title, Length: 187, dtype: object

In [23]:
empty_dataframe = pd.DataFrame(columns = ['title','artist','id','yt_title','yt_artist'])

In [24]:
spotify_track_ids = []
track_names = []

In [283]:
%%time
for track in df_yt_tracks.index:
#     print(df_yt_tracks['title'].iloc[track]+''+df_yt_tracks['artist'].iloc[track])
#     results = sp.search(q= {df_yt_tracks['title'].iloc[track]}  {df_yt_tracks['artist'].iloc[track]}" ,limit=5, type ='track')
    results = sp.search(q= '{} {}'.format(df_yt_tracks['title'].iloc[track], df_yt_tracks['artist'].iloc[track]) ,limit=5, type ='track')
#     results = sp.search(q= 'track:{}'.format(df_yt_tracks['title'].iloc[track]) ,limit=5, type ='track')
    if results['tracks']['total']>0:
        for i in results['tracks']['items']:
            i['name'] = re.sub(r'\([^()]*\)','',i['name']).strip()
#             i['artists'][0]['name'] = re.sub(r'\([^()]*\)','',i['name']).strip().capitalize()
            if fuzz.partial_ratio(i['name'], df_yt_tracks['title'].iloc[track]) >90 and fuzz.partial_ratio(i['artists'][0]['name'],df_yt_tracks['artist'].iloc[track])>60:
                spotify_track_ids.append(i['id'])
                track_names.append(df_yt_tracks['title'].iloc[track])
                break
            
        

Wall time: 32.6 s


In [278]:
%%time
for track in df_yt_tracks.index:
    results = sp.search(q= '{} {}'.format(df_yt_tracks['title'].iloc[track], df_yt_tracks['artist'].iloc[track]) ,limit=5, type ='track')
    if results['tracks']['total']>0:
        for i in results['tracks']['items']:
            empty_dataframe = empty_dataframe.append({'title':i['name'],'artist':i['artists'][0]['name'],'id':i['id'],'yt_title':df_yt_tracks['title'].iloc[track],'yt_artist':df_yt_tracks['artist'].iloc[track]},ignore_index = True) 
        empty_dataframe['title'] = empty_dataframe['title'].apply(lambda x:(re.sub(r'\([^()]*\)','',x)).strip())
#         empty_dataframe['title'] = empty_dataframe['title'].apply(lambda x:x.strip())
        empty_dataframe['fuzz_title'] = fuzz.partial_ratio(empty_dataframe['title'].all(),empty_dataframe['yt_title'].all())
        empty_dataframe['fuzz_artist'] = fuzz.partial_ratio(empty_dataframe['artist'].all(),empty_dataframe['yt_artist'].all())
#         empty_dataframe = np.where(empty_dataframe.loc[(empty_dataframe['fuzz_title'].all() > 90) & (empty_dataframe['fuzz_artist'].all() > 60)],True,False)
#         prov_df.sort_values(by=['fuzz_title','fuzz_artist'])
        prov_df = empty_dataframe.loc[((empty_dataframe['fuzz_title']) >90) & (empty_dataframe['fuzz_artist']>60)]
        if len(prov_df) > 0:
            prov_df.sort_values(by=['fuzz_artist','fuzz_title'],ascending=False,inplace = True)
            spotify_track_ids.append(prov_df['id'].iloc[0])
            track_names.append(prov_df['title'].iloc[0])
#         break
#         prov_df = empty_dataframe.loc[(fuzz.partial_ratio(empty_dataframe['title'],df_yt_tracks['title'].iloc[track]) > 90) & (fuzz.partial_ratio(empty_dataframe['artist'],df_yt_tracks['artist'].iloc[track])>60)]
#         prov_df = empty_dataframe.loc[(fuzz.partial_ratio(empty_dataframe['title'],empty_dataframe['yt_track']) > 90) & (fuzz.partial_ratio(empty_dataframe['artist'],empty_dataframe['yt_artist'])>60)]
#         prov_df = empty_dataframe[(fuzz.partial_ratio(empty_dataframe['title'],df_yt_tracks['title'].iloc[track]) > 90) & (fuzz.partial_ratio(empty_dataframe['artist'],df_yt_tracks['artist'].iloc[track])>60)]
#         prov_df = empty_dataframe.loc[(empty_dataframe.apply(lambda row: fuzz.partial_ratio(row['title'],df_yt_tracks['title'].iloc[track])) > 90)]
#         spotify_track_ids.append(prov_df['id'].iloc[0])
#         track_names.append(empty_dataframe['title'].iloc[4])


Wall time: 43.1 s


In [305]:

for track in df_yt_tracks.index:
    results = sp.search(q= 'artist:{}'.format(df_yt_tracks['artist'].iloc[track]) ,limit=5, type ='artist')
    print(results['artists']['items'])
    break
#     results = sp.search(q= 'track:{}'.format(df_yt_tracks['title'].iloc[track]) ,limit=5, type ='track')
#     if results['tracks']['total']>0:
#         for i in results['tracks']['items']:
#             i['name'] = re.sub(r'\([^()]*\)','',i['name']).strip()
# #             i['artists'][0]['name'] = re.sub(r'\([^()]*\)','',i['name']).strip().capitalize()
#             if fuzz.partial_ratio(i['name'], df_yt_tracks['title'].iloc[track]) >90 and fuzz.partial_ratio(i['artists'][0]['name'],df_yt_tracks['artist'].iloc[track])>60:
#                 spotify_track_ids.append(i['id'])
#                 track_names.append(df_yt_tracks['title'].iloc[track])
#                 break

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Y3xIwWyq5wnNHPp5gPjOW'}, 'followers': {'href': None, 'total': 673315}, 'genres': ['album rock', 'classic rock', 'mellow gold', 'new wave pop', 'rock', 'soft rock', 'yacht rock'], 'href': 'https://api.spotify.com/v1/artists/3Y3xIwWyq5wnNHPp5gPjOW', 'id': '3Y3xIwWyq5wnNHPp5gPjOW', 'images': [{'height': 1503, 'url': 'https://i.scdn.co/image/9fff0f365d90fd652a3c53f9f962676a4c2edbe3', 'width': 1000}, {'height': 962, 'url': 'https://i.scdn.co/image/ad7cc47828a040f6a90f90081cd7864f0d78e6fc', 'width': 640}, {'height': 301, 'url': 'https://i.scdn.co/image/8d279b0131aeb7c7f9b503d4be9371af06033154', 'width': 200}, {'height': 96, 'url': 'https://i.scdn.co/image/a7f37fed01316dca52c9dec0839e89786f0ea845', 'width': 64}], 'name': 'Kenny Loggins', 'popularity': 72, 'type': 'artist', 'uri': 'spotify:artist:3Y3xIwWyq5wnNHPp5gPjOW'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/0qqMTWQ3xe6MJX3wJehNTh'}, 'followers': {'href'

In [46]:
df_yt_tracks['test_sp_song'] = sp.search(q = ("{} {}".format(df_yt_tracks['title'], df_yt_tracks['artist'])),limit = 5, type = 'track')['tracks']['items']

ValueError: Length of values does not match length of index

In [ ]:
prov_df = empty_dataframe[(empty_dataframe.apply(lambda row: fuzz.partial_ratio(row['title'],df_yt_tracks['title'].iloc[track]), axis=1) > 90) & (empty_dataframe.apply(lambda row: fuzz.partial_ratio(row['artist'],df_yt_tracks['artist'].iloc[track]), axis=1) > 60)]

In [284]:
len(spotify_track_ids)

186

In [291]:
set(df_yt_tracks['title']) - set(track_names)

{'How to Prepare for IELTS!'}

In [298]:
sp.search(q = ('artist:Queen Don\'t Stop Me Now '), limit = 5, type = 'artist')

{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3AQueen+Don%27t+Stop+Me+Now+&type=artist&offset=0&limit=5',
  'items': [],
  'limit': 5,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [32]:
df_yt_tracks_test = df_yt_tracks.copy()

In [33]:
df_yt_tracks_test['title'] = df_yt_tracks['title'].apply(lambda x:re.sub(r'\([^()]*\)','',x))
df_yt_tracks['title'] = df_yt_tracks['title'].apply(lambda x:x.strip())

In [72]:
fuzz.token_set_ratio('Fun','fun.')

100

In [48]:
df_yt_tracks['title']+df_yt_tracks['artist']

0                 FootlooseKenny Loggins
1         Just the Way You AreBilly Joel
2            One SummerDaryl Braithwaite
3            Brown Eyed GirlVan Morrison
4                    SacrificeElton John
                     ...                
182            All Out of LoveAir Supply
183                   SatisfyaImran Khan
184                Mess Is MineVance Joy
185                   Let It GoJames Bay
186    How to Prepare for IELTS!E2 IELTS
Length: 187, dtype: object

In [47]:
"{} {}".format(df_yt_tracks['title'], df_yt_tracks['artist'])

'0                      Footloose\n1           Just the Way You Are\n2                     One Summer\n3                Brown Eyed Girl\n4                      Sacrifice\n                 ...            \n182              All Out of Love\n183                     Satisfya\n184                 Mess Is Mine\n185                    Let It Go\n186    How to Prepare for IELTS!\nName: title, Length: 187, dtype: object 0          Kenny Loggins\n1             Billy Joel\n2      Daryl Braithwaite\n3           Van Morrison\n4             Elton John\n             ...        \n182           Air Supply\n183           Imran Khan\n184            Vance Joy\n185            James Bay\n186             E2 IELTS\nName: artist, Length: 187, dtype: object'